In [ ]:
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K
import numpy as np
%matplotlib inline
from matplotlib import pyplot as plt
from my_classes import DataGenerator

Read in the data

In [ ]:
FFP = True
ffpname = ''
if FFP:
    ffpname='_kp'

batch_size = 128
num_classes = 2
input_shape = (128,128)

In [ ]:
params = {'dim': (128,128),
          'batch_size': batch_size,
          'n_classes': num_classes,
          'n_channels': 1,
          'shuffle': True}

# Datasets
np.load('partition.npy')
np.load('labels.npy')


#partition = # IDs
#labels = # Labels

# Generators
training_generator = DataGenerator(partition['train'], labels, **params)
validation_generator = DataGenerator(partition['validation'], labels, **params)


Reformatting data for keras

In [ ]:
#if K.image_data_format() == 'channels_first':

Build the network

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

A summary of the network structure

In [ ]:
model.summary()

Train the network

In [ ]:
modelname = 'PB19_network_5.dms'
if FFP:
    modelname = 'PB19_FFP_network_5.dms'

epochs = 12
model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit_generator(generator=training_generator,
                    validation_data=validation_generator,
                    use_multiprocessing=True,
                    workers=6)

model.save(modelname)